In [2]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers

In [3]:
# xi | ti ~ N(logit(alpha*ti), (beta*ti)^2)
# ti ~ N(0,Sigma)

N, d = int(1e6), 3
alpha = np.linspace(1,d,d)
beta = np.random.randint(10, size=d) - 5

def data(N, d, alpha, beta):
    X = np.random.normal(size = (int(N/1000), d))
    X = np.concatenate([X for i in range(1000)], axis=0)
    muiY = np.exp(X.dot(alpha)) / (1 + np.exp(X.dot(alpha)))
    sigY = X.dot(beta)
    preY = np.random.normal(size = (N,))
    Y = preY * sigY + muiY
    return X, Y

X, Y = data(N,d,alpha,beta)
Xval, Yval = data(N,d,alpha,beta)

Xval.shape, Yval.shape

((1000000, 3), (1000000,))

In [6]:
tf.keras.backend.clear_session()

def densityEncoder(dim):
    inputs = tf.keras.Input(shape=(dim,))
    
    mui = layers.Dense(1, activation='sigmoid')(inputs)
    
#     sig = layers.Dense(1)(inputs)
    
#     outputs = tf.concat([mui,sig],axis=1)
    outputs = mui
    return tf.keras.Model(inputs=inputs, outputs=outputs)

def densityLoss(Y,theta):
    mui, sig2 = theta[:,0], theta[:,-1]**2
    negloglike = 0.5 * (tf.math.log(sig2) + (Y-mui)**2 / sig2)
    return tf.reduce_mean(negloglike)
    
model = densityEncoder(d)
model.compile(optimizer=tf.keras.optimizers.Adam(lr=1e-2),
              loss='mse')
model.summary()

model.fit(X, Y, batch_size=int(1e4), epochs=20,
          validation_data = (Xval, Yval))

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 3)]               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 4         
Total params: 4
Trainable params: 4
Non-trainable params: 0
_________________________________________________________________
Train on 1000000 samples, validate on 1000000 samples
Epoch 1/20
1000000/1000000 [==============================] - 1s 1us/sample - loss: 12.3433 - val_loss: 13.1765
Epoch 2/20
1000000/1000000 [==============================] - 1s 1us/sample - loss: 12.3281 - val_loss: 13.1707
Epoch 3/20
1000000/1000000 [==============================] - 1s 1us/sample - loss: 12.3247 - val_loss: 13.1689
Epoch 4/20
1000000/1000000 [==============================] - 1s 1us/sample - loss: 12.3234 - val_loss: 13.1683
Epoch 5/20
1000000/1000000 

In [7]:
Xval_para = model.predict(Xval)

muiYval = np.exp(Xval.dot(alpha)) / (1 + np.exp(Xval.dot(alpha)))
sigYval = Xval.dot(beta)

In [8]:
Xval_para[1:10,:], muiYval[1:10], sigYval[1:10]

(array([[9.6435416e-01],
        [9.3344343e-01],
        [9.9034351e-01],
        [6.6751242e-04],
        [1.7700875e-01],
        [1.1261731e-02],
        [2.2615945e-01],
        [9.7642219e-01],
        [9.8162240e-01]], dtype=float32),
 array([9.63022087e-01, 9.30218152e-01, 9.89979999e-01, 7.21630357e-04,
        1.80301730e-01, 1.11343758e-02, 2.36835860e-01, 9.75502034e-01,
        9.81351350e-01]),
 array([-3.37918818, -2.85408804, -4.45865288,  7.07501953,  1.71312921,
         1.82563391,  2.41471635, -4.73571953, -3.49374167]))